In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [2]:
# import pytesseract
# local_path = "Arunanshu Resume.pdf"
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# # Local PDF file uploads
# if local_path:
#   loader = UnstructuredPDFLoader(file_path=local_path)
#   data = loader.load()
# else:
#   print("Upload a PDF file")

In [21]:
import PyPDF2
import io

def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

local_path = "hil-serl-paper.pdf"

if local_path:
    extracted_text = extract_text_from_pdf(local_path)
    print("Extracted text:", extracted_text[:500])  # Print first 500 characters
else:
    print("Upload a PDF file")

Extracted text: Oct 2024
Precise and Dexterous Robotic Manipulation via
Human-in-the-Loop Reinforcement Learning
Jianlan Luo1, Charles Xu1, Jeffrey Wu1and Sergey Levine1
1Department of Electrical Engineering and Computer Sciences, UC Berkeley
Reinforcement learning (RL) holds great promise for enabling autonomous acquisition of complex robotic
manipulation skills, but realizing this potential in real-world settings has been challenging. We present a
human-in-the-loop vision-based RL system that demonstrates imp


In [22]:
print("Extracted text:", extracted_text[:1000])

Extracted text: Oct 2024
Precise and Dexterous Robotic Manipulation via
Human-in-the-Loop Reinforcement Learning
Jianlan Luo1, Charles Xu1, Jeffrey Wu1and Sergey Levine1
1Department of Electrical Engineering and Computer Sciences, UC Berkeley
Reinforcement learning (RL) holds great promise for enabling autonomous acquisition of complex robotic
manipulation skills, but realizing this potential in real-world settings has been challenging. We present a
human-in-the-loop vision-based RL system that demonstrates impressive performance on a diverse set
of dexterous manipulation tasks, including dynamic manipulation, precision assembly, and dual-arm
coordination. Our approach integrates demonstrations and human corrections, efficient RL algorithms,
and other system-level design choices to learn policies that achieve near-perfect success rates and fast cycle
times within just 1 to 2.5 hours of training. We show that our method significantly outperforms imitation
learning baselines and prior RL

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_text(extracted_text)
print(f"Number of chunks: {len(chunks)}")
print("First chunk:", chunks[0][:200])

Number of chunks: 19
First chunk: Oct 2024
Precise and Dexterous Robotic Manipulation via
Human-in-the-Loop Reinforcement Learning
Jianlan Luo1, Charles Xu1, Jeffrey Wu1and Sergey Levine1
1Department of Electrical Engineering and Comp


In [4]:
from langchain.docstore.document import Document
import os


documents = [Document(page_content=chunk) for chunk in chunks]

persist_directory = "chroma_db_akshay_hehe"

# Ensure the directory exists
os.makedirs(persist_directory, exist_ok=True)

vector_db = Chroma.from_documents(
    documents=documents,
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    persist_directory=persist_directory,
    collection_name="local-rag"
)

print(f"Number of documents in Chroma: {vector_db._collection.count()}")

NameError: name 'chunks' is not defined

In [1]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [2]:
local_model = "llama3.1"
llm = ChatOllama(model=local_model)

C:\Users\aruna\AppData\Local\Temp\ipykernel_10400\1884553848.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_model)


In [3]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [18]:
# if already extracted the vector db - 
persist_directory = "chroma_db_5_tb"
vector_db = Chroma(
    persist_directory=persist_directory,
    embedding_function=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)

In [19]:
retriever = MultiQueryRetriever.from_llm(
    retriever=vector_db.as_retriever(search_kwargs={"k": 2}),
    llm=llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [20]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
# chain.invoke("Tell me about the Education background of the applicant based on the document provided")

In [ ]:
chain.invoke("A 45-year-old patient presents with persistent dry cough, fatigue, low-grade fever, and chest discomfort for the past two weeks. What could be the possible diagnosis, the recommended medication, and the severity of the condition?")

"Based on the symptoms described (persistent dry cough, fatigue, low-grade fever, and chest discomfort), it is likely that the patient has a respiratory infection.\n\nPossible Diagnosis:\n\n*   Pneumonia: This is an inflammation of the lung tissue caused by an infection. The symptoms described are consistent with pneumonia.\n*   Tuberculosis: This is a bacterial infection that can cause symptoms similar to pneumonia.\n*   Bronchitis: This is an inflammation of the bronchial tubes, which can also present with coughing and chest discomfort.\n\nRecommended Medication:\n\nThe recommended medication would depend on the specific diagnosis. However, for pneumonia or tuberculosis, the following medications may be used:\n\n*   Antibiotics (e.g., ciprofloxacin, azithromycin) for pneumonia\n*   Antituberculosis drugs (e.g., isoniazid, rifampin) for tuberculosis\n\nSeverity of Condition:\n\nThe severity of the condition would depend on the specific diagnosis and the patient's overall health. Howev

In [23]:
chain.invoke("A 60-year-old patient with a history of high blood pressure and diabetes reports sudden onset of sharp chest pain, shortness of breath, sweating, and dizziness. What is the likely diagnosis, recommended treatment, and the severity of the condition?")

'Based on the symptoms described (sudden onset of sharp chest pain, shortness of breath, sweating, and dizziness), it appears that the patient may be experiencing an acute cardiac event, specifically a myocardial infarction (MI).\n\n**Likely Diagnosis:** Acute Myocardial Infarction (MI) or Heart Attack\n\nMyocardial infarction occurs when blood flow to the heart muscle is blocked for a long enough time that it causes damage to the heart muscle. This condition can cause severe chest pain, shortness of breath, sweating, and dizziness.\n\n**Recommended Treatment:**\n\nThe recommended treatment for MI involves immediate medical attention, usually in an emergency department or cardiac catheterization lab. The goal of treatment is to restore blood flow to the affected area of the heart as quickly as possible.\n\n1. **Aspirin:** Administered immediately to help prevent further clotting.\n2. **Oxygen Therapy:** Provided to ensure adequate oxygenation.\n3. **Nitroglycerin:** May be given to hel

In [13]:
chain.invoke("Frame 10 questions along with answers from the document.")

'Here are the 10 questions along with answers based on the provided text:\n\n**1. What is the recommended daily caloric intake for patients with severe TBI?**\nAnswer: Approximately 140% of expected requirements.\n\n**2. How often should posttraumatic seizures be treated with anticonvulsants?**\nAnswer: For 7 days after TBI.\n\n**3. What is the goal mean arterial pressure (MAP) for patients with severe TBI?**\nAnswer: Above 80 mm Hg until cerebral perfusion pressure (CPP) can be measured.\n\n**4. What should be done to control ICP according to Algorithm 60.1?**\nAnswer: See Algorithm 60.1 for detailed steps.\n\n**5. Why should antimicrobials not be delayed in the treatment of CNS infections?**\nAnswer: To prevent devastating complications and ensure prompt initiation of effective therapy.\n\n**6. What is the recommended duration for seizure prophylaxis after TBI?**\nAnswer: Not to extend beyond 1 week.\n\n**7. How often should patients with severe TBI receive nutritional support?**\nAn

In [14]:
chain.invoke("Which textbook does the document originate from?")

'The document appears to be a chapter or section from a medical textbook, likely "Critical Care Medicine: A Case-Focused Approach" given the table numbers and chapter titles that match this book. However, without more information, it\'s difficult to pinpoint the exact title of the original textbook.'

In [20]:
chain.invoke("Anything crucial in the document apart from Experience Replay, Target Network and Error Clipping?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


"Yes, there are a few more important concepts mentioned in the document:\n\n1. **Experience Replay**: This is a technique used to improve the efficiency of training deep Q-networks (DQN) by storing experiences (states, actions, rewards, next states, etc.) in a replay buffer and sampling from this buffer during training.\n2. **Target Network**: In DQN, there are two networks: the **Q-Network** (also called the prediction network) which predicts the action values Q(s,a), and the **Target Network** which is used to compute the target values y = r + γmaxa' Q(s', a'; θ'-). The Target Network is updated periodically (e.g., every C steps) by copying the weights of the Q-Network.\n3. **Error Clipping**: This is a technique used to prevent exploding gradients in DQN by clipping the error term (the difference between predicted and target values) to a certain range (-c, c).\n\nThese three components are crucial for stabilizing the training process and improving the performance of deep Q-networks.

In [19]:
!pip install unsloth "xformers==0.0.28.post2"

  Using cached unsloth-2024.11.5-py3-none-any.whl (161 kB)
  Using cached xformers-0.0.28.post2.tar.gz (7.8 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\aruna\\AppData\\Local\\Temp\\pip-install-lfvaacbw\\xformers_9d5f59bd5a3e42099233fded4dc40f44\\third_party/flash-attention/csrc/composable_kernel/client_example/24_grouped_conv_activation/grouped_convnd_bwd_data_bilinear/grouped_conv_bwd_data_bilinear_residual_fp16.cpp'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
